In [393]:
import pandas as pd, numpy as np

# Ex04. Обогащение и преобразования

## 1. Чтение файла auto.json

In [394]:
df = pd.read_json("data/auto.json")
pd.options.display.float_format = '{:.2f}'.format

## 2. Создание выборки 

### 2.1. Создание выборки из исходного фрейма данных с сохранением комбинаций car number, make, и model

In [395]:
car_combinations = df[['CarNumber', 'Make', 'Model']].drop_duplicates().sample(n=200, ignore_index=True)
fines_ = df[['Fines']].drop_duplicates().sample(n=200, replace=True, ignore_index=True)
refund_ = df[['Refund']].drop_duplicates().sample(n=200, replace=True, ignore_index=True)

sample_df = pd.concat([car_combinations, fines_, refund_], axis=1)
sample_df

,CarNumber,Make,Model,Fines,Refund
0,H906YK197RUS,Ford,Focus,21600.00,2
1,O058M7161RUS,Ford,Focus,12000.00,2
2,7193TT36RUS,Ford,Focus,5600.00,1
3,9499KX178RUS,Ford,Focus,2700.00,1
4,H427YH197RUS,Volkswagen,Passat,13500.00,1
...,...,...,...,...,...
195,Y353O8197RUS,Ford,Focus,8500.00,1
196,O68897197RUS,Ford,Focus,5000.00,2
197,8441XX154RUS,Ford,Focus,10300.00,2
198,Y358O8197RUS,Ford,Focus,63300.00,2


**Проверка на наличие новых комбинаций car number, make, и model**

In [396]:
mask = ~sample_df[['CarNumber', 'Make', 'Model']].drop_duplicates().apply(tuple, axis=1).isin(
    df[['CarNumber', 'Make', 'Model']].drop_duplicates().apply(tuple, axis=1)
)
new_combos = sample_df[mask]
print("Количество новых комбинаций:", len(new_combos))
if len(new_combos) > 0:
    print(new_combos)
else:
    print("Все комбинации уже были в исходном датафрейме.")

Количество новых комбинаций: 0
Все комбинации уже были в исходном датафрейме.


### 2.2. Обьединение исходного фрейма данных с выборкой

In [397]:
concat_rows = pd.concat([df, sample_df], ignore_index=True)
concat_rows

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus
...,...,...,...,...,...
920,Y353O8197RUS,1,8500.00,Ford,Focus
921,O68897197RUS,2,5000.00,Ford,Focus
922,8441XX154RUS,2,10300.00,Ford,Focus
923,Y358O8197RUS,2,63300.00,Ford,Focus


## 3. Создание нового столбца year и обьединение фрейма данных с ним

In [398]:
np.random.seed(21)
years = pd.Series(np.random.randint(1980, 2019, size=len(concat_rows)))
fines = concat_rows.assign(Year=years)
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
920,Y353O8197RUS,1,8500.00,Ford,Focus,1996
921,O68897197RUS,2,5000.00,Ford,Focus,2002
922,8441XX154RUS,2,10300.00,Ford,Focus,1996
923,Y358O8197RUS,2,63300.00,Ford,Focus,2012


## 4. Обогащение фрейма данных данными из другого фрейма данных

### 4.1. Чтение файла surname.json

In [399]:
surname_df = pd.read_json("data/surname.json")
surname_df.columns = surname_df.iloc[0]
surname_df = surname_df[1:].reset_index(drop=True)

### 4.2. Самые популярные фамилии в США

In [400]:
surname_df.loc[surname_df['COUNT'].idxmax()]['NAME']

'LOPEZ'

### 4.3. Создание нового фрейма данных с фамилиями

In [401]:
unique_car_numbers = df['CarNumber'].drop_duplicates(ignore_index=True)
names_sample = surname_df['NAME'].sample(n = len(unique_car_numbers), random_state=21, replace=True, ignore_index=True)
owners = pd.concat([unique_car_numbers, names_sample], axis=1).rename(columns = {'NAME': 'SURNAME'})
owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
526,O136HO197RUS,CAMPBELL
527,O22097197RUS,HALL
528,M0309X197RUS,BAKER
529,O673E8197RUS,DIAZ


### 4.4. Добавление новых наблюдений в fines 

In [402]:
new_fines_rows = pd.DataFrame([
    ['C113TH777RUS', 2, 15000, 'BMW', '3', 2023],
    ['H345TK777RUS', 2, 1000, 'Mercedes', 'E63', 2018],
    ['C123TH777RUS', 1, 18000, 'BMW', '3', 2023],
    ['C133TH777RUS', 2, 5000, 'BMW', '3', 2023],
    ['H335TK777RUS', 2, 11000, 'Mercedes', 'E63', 2018]
])
new_fines_rows.columns = fines.columns
fines = pd.concat([fines, new_fines_rows], ignore_index=True)
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
925,C113TH777RUS,2,15000.00,BMW,3,2023
926,H345TK777RUS,2,1000.00,Mercedes,E63,2018
927,C123TH777RUS,1,18000.00,BMW,3,2023
928,C133TH777RUS,2,5000.00,BMW,3,2023


### 4.5. Удаление старых и добавление новых строк в owners

In [403]:
owners = owners.iloc[:-20]

In [404]:
new_owners_rows = pd.DataFrame([
    ['X777XX777RUS', 'APPLE'],
    ['P737XP777RUS', 'MACINTOSH'],
    ['K768KH777RUS', 'STELLA'],
])
new_owners_rows.columns = owners.columns
owners = pd.concat([owners, new_owners_rows], ignore_index=True)
owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
509,O50197197RUS,WRIGHT
510,7608EE777RUS,HILL
511,X777XX777RUS,APPLE
512,P737XP777RUS,MACINTOSH


### 4.6. Объединение фреймов по столбцу CarNumber

**Inner join**

In [405]:
pd.merge(fines, owners, on='CarNumber')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
894,Y353O8197RUS,1,8500.00,Ford,Focus,1996,WHITE
895,O68897197RUS,2,5000.00,Ford,Focus,2002,MITCHELL
896,8441XX154RUS,2,10300.00,Ford,Focus,1996,MOORE
897,Y358O8197RUS,2,63300.00,Ford,Focus,2012,ROBINSON


**Outer join**

In [406]:
pd.merge(fines, owners, on='CarNumber', how='outer')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,704687163RUS,2.00,1400.00,Ford,Focus,2014.00,ADAMS
1,704787163RUS,2.00,2800.00,Ford,Focus,2005.00,MORGAN
2,704787163RUS,2.00,6500.00,Ford,Focus,2001.00,MORGAN
3,704987163RUS,2.00,8594.59,Ford,Focus,2014.00,MITCHELL
4,704987163RUS,1.00,4200.00,Ford,Focus,2000.00,MITCHELL
...,...,...,...,...,...,...,...
928,Y969O8197RUS,2.00,7800.00,Ford,Focus,1987.00,LOPEZ
929,Y973O8197RUS,2.00,8594.59,Ford,Focus,2005.00,YOUNG
930,Y973O8197RUS,1.00,34800.00,Ford,Focus,2013.00,YOUNG
931,Y973O8197RUS,1.00,69600.00,Ford,Focus,1989.00,YOUNG


**Left join**

In [407]:
pd.merge(fines, owners, on='CarNumber', how='left')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
925,C113TH777RUS,2,15000.00,BMW,3,2023,NaN
926,H345TK777RUS,2,1000.00,Mercedes,E63,2018,NaN
927,C123TH777RUS,1,18000.00,BMW,3,2023,NaN
928,C133TH777RUS,2,5000.00,BMW,3,2023,NaN


**Right join**

In [408]:
pd.merge(fines, owners, on='CarNumber', how='right')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2.00,3200.00,Ford,Focus,1989.00,RICHARDSON
1,Y163O8161RUS,2.00,1600.00,Ford,Focus,1999.00,RICHARDSON
2,E432XX77RUS,1.00,6500.00,Toyota,Camry,1995.00,ROSS
3,E432XX77RUS,2.00,13000.00,Toyota,Camry,1992.00,ROSS
4,E432XX77RUS,1.00,29700.00,Toyota,Camry,2008.00,ROSS
...,...,...,...,...,...,...,...
897,O50197197RUS,2.00,7800.00,Ford,Focus,1986.00,WRIGHT
898,7608EE777RUS,1.00,4000.00,Skoda,Octavia,2013.00,HILL
899,X777XX777RUS,NaN,NaN,NaN,NaN,NaN,APPLE
900,P737XP777RUS,NaN,NaN,NaN,NaN,NaN,MACINTOSH


## 5. Сводная таблица штрафов по годам

In [409]:
fines_copy = fines.copy()
fines_copy[['Make', 'Model']] = fines[['Make', 'Model']].fillna(' ')
fines_copy.pivot_table(
    index=['Make', 'Model'],
    columns='Year',
    values='Fines',
    aggfunc='sum',
    fill_value='-'
)

Year                    1980      1981      1982      1983      1984  \
Make       Model                                                       
Audi                       -         -         -         -         -   
BMW                        -         -         -         -   3800.00   
           3               -         -         -         -         -   
Ford       Focus   156400.00 415694.59 127200.00 134694.59 179800.00   
           Mondeo          -         -  46200.00         -         -   
Mercedes   E63             -         -         -         -         -   
Skoda      Octavia   8594.59  10100.00   8894.59         -    300.00   
Toyota     Camry    12000.00         -         -   1600.00   1000.00   
           Corolla         -   6800.00         -  12800.00         -   
Volkswagen           1300.00   7900.00         -         -         -   
           Golf     20800.00   8594.59   5000.00    200.00         -   
           Jetta           -   1000.00         -  22400.00         -   
           Passat     900.00  40800.00   9900.00   1100.00   8594.59   
           Touareg         -         -         -         -         -   
Volvo                      -         -         -         -         -   

Year                    1985     1986      1987      1988      1989  ...  \
Make       Model                                                     ...   
Audi                       -        -         -         -         -  ...   
BMW                        -        -         -         -         -  ...   
           3               -        -         -         -         -  ...   
Ford       Focus   295994.59 59200.00 129294.59 158478.35 211394.59  ...   
           Mondeo          -        -         -         -         -  ...   
Mercedes   E63             -        -         -         -         -  ...   
Skoda      Octavia  20594.59 11000.00   2000.00   5100.00   8594.59  ...   
Toyota     Camry           - 19800.00         -         -    800.00  ...   
           Corolla         -        -  54300.00         -   7800.00  ...   
Volkswagen                 -  7400.00         -         -         -  ...   
           Golf    168000.00        -         -         -  34300.00  ...   
           Jetta     9000.00        -         -  46000.00         -  ...   
           Passat          - 16000.00   2000.00   8594.59         -  ...   
           Touareg         -        -         -         -         -  ...   
Volvo                      -        -         -         -         -  ...   

Year                    2010      2011      2012      2013     2014      2015  \
Make       Model                                                                
Audi                       -         -         -         -        -         -   
BMW                        -   8594.59         -         -        -         -   
           3               -         -         -         -        -         -   
Ford       Focus   146100.00 146094.59 205589.17 281689.17 85383.76 287700.00   
           Mondeo          -         -         -  44400.00  8000.00         -   
Mercedes   E63             -         -         -         -        -         -   
Skoda      Octavia   2500.00   5600.00   1700.00  11800.00 41700.00  16394.59   
Toyota     Camry    22400.00         -   7500.00         -        -         -   
           Corolla   6000.00         -         -         -        -   1800.00   
Volkswagen                 -         -         -         -        -         -   
           Golf            -         -   2500.00         - 26800.00         -   
           Jetta           -         -         -         -        -         -   
           Passat   25300.00         -         -  30900.00        -         -   
           Touareg         -         -         -         -        -         -   
Volvo                      -         -         -         -        -  10200.00   

Year                    2016      2017      2018     2023  
Make       Model                            

## 6. Сохранение фреймов данных в файлы csv

In [410]:
fines.to_csv("data/fines.csv", sep=',', index=False, float_format='%.2f')
owners.to_csv("data/owners.csv", sep=',', index=False)